# SFv2 Template notebook

## Import modules

In [16]:
import os
import sys

data_path=os.path.abspath('C:/Users/hcarv/Documents/work/Protein Design 2021/Group1') #Where your group data is
base_path=os.path.abspath('C:/Users/hcarv/Documents/GitHub/SimFound_v2/source/') #Where your source code is (SFv2)

sys.path.append(base_path)
sys.path.append(data_path)

import importlib

import Protocols as P


# Protocols

In [26]:
pdb='5a71a.pdb'

importlib.reload(P)
sim_tools=P.Protocols(workdir=data_path)

system_omm=sim_tools.pdb2omm(input_pdbs=pdb, 
                            extra_ff_files=['gaff-2.11modv3_1.xml'], 
                            protonate=True,
                            solvate=True,
                            inspect=True,
                            pH=8.0) 

Extra force field file 1: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/gaff-2.11modv3_1.xml
Using default force fields: ('amber14-all.xml', 'amber14/tip4pew.xml')
PDB file generated: C:\Users\hcarv\Documents\work\Protein Design 2021\Group1/system.pdb


In [27]:
system_omm.setSimulation()

In [ ]:
system_omm.energyMinimization()

Performing energy minimization: -738450.583629637 kJ/mol


In [ ]:
EQ=system.equilibration_NPT()